In [1]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:05<00:00, 15.3MB/s]


In [3]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [4]:
print(vars(train_data.examples[0]))

{'text': ['This', 'film', 'is', 'about', 'a', 'man', 'who', 'has', 'been', 'too', 'caught', 'up', 'with', 'the', 'accepted', 'convention', 'of', 'success', ',', 'trying', 'to', 'be', 'ever', 'upwardly', 'mobile', ',', 'working', 'hard', 'so', 'that', 'he', 'could', 'be', 'proud', 'of', 'owning', 'his', 'own', 'home', '.', 'He', 'assumes', 'this', 'is', 'all', 'there', 'is', 'to', 'life', 'until', 'he', 'accidentally', 'takes', 'up', 'dancing', ',', 'all', 'because', 'he', 'wanted', 'to', 'get', 'a', 'closer', 'look', 'of', 'a', 'beautiful', 'girl', 'that', 'he', 'sees', 'by', 'the', 'dance', 'studio', 'everyday', 'while', 'riding', 'the', 'subway', 'on', 'his', 'way', 'home.<br', '/><br', '/>His', 'was', 'infatuated', 'with', 'her', 'at', 'first', ',', 'going', 'to', 'the', 'dance', 'class', 'just', 'to', 'idolize', 'her', ',', 'but', 'he', 'eventually', 'lets', 'himself', 'go', 'and', 'gets', 'himself', 'into', 'the', 'dancing', '.', 'It', 'eventually', 'becomes', 'apparent', 'to', 'h

In [5]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [6]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [7]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [8]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202412), (',', 192759), ('.', 165751), ('and', 109636), ('a', 109396), ('of', 100712), ('to', 93894), ('is', 76323), ('in', 60993), ('I', 54132), ('it', 53828), ('that', 48896), ('"', 44196), ("'s", 43234), ('this', 42358), ('-', 36941), ('/><br', 35289), ('was', 34903), ('as', 30155), ('with', 29886)]


In [9]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [10]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [11]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [12]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [19]:
print('INPUT_DIM = ',len(TEXT.vocab))

INPUT_DIM =  25002


In [20]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [21]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [22]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [23]:
criterion = nn.BCEWithLogitsLoss()

In [24]:
model = model.to(device)
criterion = criterion.to(device)

In [25]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [26]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [27]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [28]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [29]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 15m 34s
	Train Loss: 0.694 | Train Acc: 49.96%
	 Val. Loss: 0.697 |  Val. Acc: 49.54%
Epoch: 02 | Epoch Time: 15m 8s
	Train Loss: 0.693 | Train Acc: 49.90%
	 Val. Loss: 0.697 |  Val. Acc: 49.60%
Epoch: 03 | Epoch Time: 15m 9s
	Train Loss: 0.693 | Train Acc: 49.99%
	 Val. Loss: 0.697 |  Val. Acc: 50.06%
Epoch: 04 | Epoch Time: 15m 25s
	Train Loss: 0.693 | Train Acc: 49.63%
	 Val. Loss: 0.697 |  Val. Acc: 49.11%
Epoch: 05 | Epoch Time: 15m 27s
	Train Loss: 0.693 | Train Acc: 50.42%
	 Val. Loss: 0.697 |  Val. Acc: 50.03%


In [30]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.710 | Test Acc: 46.04%
